In [1]:
from comet_ml import Experiment
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
import torch
from torchsummary import summary
import numpy as np
from tqdm import tqdm
torch.cuda.set_device(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
from torch.utils.data import Dataset, DataLoader

In [2]:
# stage should be in 0 to 5 and for 0, use -1 (this is due to inconsistency in the model generated by PyTorch)
hyper_params = {
    "stage": 0,
    "repeated": 1,
    "num_classes": 10,
    "batch_size": 64,
    "num_epochs": 1,
    "learning_rate": 1e-4
}

In [3]:
def save_torch(name:str, tensor):
    new = tensor.clone()
    np.save(name, new.detach().cpu().numpy())
    
def load_np_torch(name):
    return torch.from_numpy(np.load(str(name)))

def save_features(model, data, stage):
    for i in tqdm(range(len(data.train_ds))):
        pred = model(data.train_ds[i][0].data.cuda().unsqueeze(0))
        save_torch(path_tn_saved_feat/f'{stage}/{data.train_ds.names[i]}.npy', pred)
        
    for i in tqdm(range(len(data.valid_ds))):
        pred = model(data.valid_ds[i][0].data.cuda().unsqueeze(0))
        save_torch(path_val_saved_feat/f'{stage}/{data.valid_ds.names[i]}.npy', pred)

In [4]:
path = untar_data(URLs.IMAGENETTE)

In [5]:
path_train_feat = path/'train_feat'
path_val_feat = path/'val_feat'

path_tn_saved_feat = path/'train_saved_feat'
path_val_saved_feat = path/'val_saved_feat'


model_save_dir = path/f'saved_models'
model_save_dir.mkdir(exist_ok=True)

path_tn_saved_feat.mkdir(exist_ok=True)
path_val_saved_feat.mkdir(exist_ok=True)

for i in range(1, 6):
    (path_tn_saved_feat/str(i)).mkdir(exist_ok=True)
    (path_val_saved_feat/str(i)).mkdir(exist_ok=True)
    
# path_tn_saved_feat.ls()[1].ls()

## DATASET

In [23]:
from PIL import Image
class FeatureDataset(Dataset):
    def __init__(self, path, stage:int, train_val:str="train"):
        super().__init__()
        assert stage < 6
        if stage == 1: self.feat1 = path/f'{train_val}_feat/{stage-1}'
        else: self.feat1 = path/f'{train_val}_saved_feat/{stage-1}'
        self.feat2 = path/f'{train_val}_feat/{stage}'
        self.stage = stage
        self.label_list = os.listdir(str(path/'train'))
        self.names = list(map(lambda x: x.stem, self.feat1.ls()))
        
    def __len__(self):
        return len(self.feat1.ls())
    
    def __getitem__(self, idx):
#         if self.stage == 1: feature1 = torch.from_numpy(
#             np.array(Image.open(self.feat1.ls()[idx]).resize((224, 224))))/255
#         else: 
        feature1 = load_np_torch(self.feat1.ls()[idx]).squeeze(0)
            
        if self.stage == 6:
            feature2 = self.label_list.index((self.feat1.ls()[idx].split('/')[-1]).split('_')[0])
            feature2 = torch.tensor(feature2)
        else: 
            feature2 = load_np_torch(self.feat2.ls()[idx]).squeeze(0)
        return feature1, feature2

In [24]:
def get_datasets(stage):
    train_dataset = FeatureDataset(path, stage, "train")
    val_dataset = FeatureDataset(path, stage, "val")

    try: 
        train_dataset[1][0].shape is type(torch.Size)
        val_dataset[1][0].shape is type(torch.Size)
    except: 
        print('test failed check dataset')
    return DataBunch.create(train_dataset, val_dataset, bs=64)

## Network 

In [25]:
class Flatten(nn.Module) :
    def forward(self, input):
        return input.view(input.size(0), -1)

def conv2(ni, nf) : 
    return conv_layer(ni, nf, stride = 2)

class ResBlock(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv1 = conv_layer(nf,nf)
        
    def forward(self, x): 
        return (x + self.conv1(x))

def conv_and_res(ni, nf): 
    return nn.Sequential(conv2(ni, nf), ResBlock(nf))

def conv_(nf) : 
    return nn.Sequential(conv_layer(nf, nf), ResBlock(nf))

In [26]:
net = [conv_layer(3, 64, ks = 7, stride = 2, padding = 3),
                    nn.MaxPool2d(3, 2, padding = 1),
                    conv_(64),
                    conv_and_res(64, 128),
                    conv_and_res(128, 256),
                    conv_and_res(256, 512),
                    nn.Sequential(AdaptiveConcatPool2d(),
                    Flatten(),
                    nn.Linear(2 * 512, 256),
                    nn.Linear(256, hyper_params["num_classes"]))
                    ]
stages = [(1, 0), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6)]

## Training

In [35]:
nets = []
training_id = 1
save_dir = path/f'saved_models/{training_id}/'
losses_save_dir = path/f'saved_models/{training_id}/losses/'
save_dir.mkdir(exist_ok=True)
losses_save_dir.mkdir(exist_ok=True)

for stage in stages:

    data = get_datasets(stage=stage[0])
#     if stage[0] == 1: continue 
#     break 
    model = net[stage[1]].cuda()
    if stage[0] < 6:
        learn = Learner(data, model, loss_func=nn.MSELoss())
    else: 
        learn = Learner(data, model, loss_func=nn.CrossEntropyLoss(), metrics=accuracy)
        
    learn.fit_one_cycle(hyper_params["num_epochs"],hyper_params["learning_rate"])
    
    model = learn.model 
    nets.append(model)
    pd.DataFrame(learn.recorder.losses).to_csv(str(losses_save_dir/f'train_stage_{stage[0]}.csv'))
    pd.DataFrame(learn.recorder.losses).to_csv(str(losses_save_dir/f'val_stage_{stage[0]}.csv'))
    torch.save(model, str(save_dir/f'model_part_{stage[1]}.pth'))
    save_features(model, data, stage=stage[0])
    clear_output(wait=True)
    
final = nn.Sequential(*nets)
torch.save(final, str(save_dir/f'complete_model.pth'))

test failed check dataset


# end

In [36]:
data.train_ds[0][0].shape

torch.Size([64, 112, 112])

In [37]:
import time 
init = time.time()
for x in data.train_dl:
    pass
(time.time() - init )/60

0.21590638558069866

In [33]:
data.train_ds.feat1.ls()

[PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03888257_26931.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03028079_9683.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03445777_2070.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03425413_11054.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03888257_16991.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n02979186_17940.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03417042_6917.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03394916_48064.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03445777_2332.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03394916_71979.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n03425413_13702.npy'),
 PosixPath('/home/navid/.fastai/data/imagenette/train_feat/0/n01440764_10382.npy